# 02467 Computational Social Science
## Assignment 1
### Group 15

Our GitHub repo is availabe at: https://github.com/Simo067m/ComSocSci-Assignments <br>
Contribution:
- s233304 : xxxx
- s214592 : xxxx

In [2]:
# Import packages
import pandas as pd
import networkx
import netwulf
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import ast

## Part 1: Web-scraping
Web-scraping the list of participants to the International Conference in Computational Social Science

In [3]:
# Define a function for finding all unique researchers
def scrape_IC2S2(soup : BeautifulSoup):
    # Find all the names from the top table
    names = []
    # Find all the table rows
    table_rows = soup.find_all("tr")
    for tr in table_rows:
        tds = tr.find_all("td")
        for row in tds:
            a = row.find_all("a")
            for text in a:
                text_content = text.text
                if ("Keynote" in text_content):
                    text_split = text_content.split("-")
                    stripped = text_split[1].strip()
                    if (stripped not in names):
                        names.append(stripped)
    
    # Find all the names from the bottom lists
    # Find all the unordered lists
    ul = soup.find_all("ul", class_="nav_list")
    # Find all the list elements
    for list in ul:
        found_names = list.find_all("i")
        # For every found name line, seperate into individual names
        for name in found_names:
            found_names_seperated = name.text.split(", ")
            for seperated_name in found_names_seperated:
                if (seperated_name.strip() not in names):
                    names.append(seperated_name.strip())

    # Find all the names of the chairs
    headers = soup.find_all("h2")
    for header in headers:
        text = header.find("i")
        if (text is not None):
            seperated_name = text.text.split(": ")
            if (seperated_name[1].strip() not in names):
                names.append(seperated_name[1].strip())

    return names

In [4]:
# Define url and collect content
LINK = "https://ic2s2-2023.org/program"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)

# find participant names
IC2S2_names = scrape_IC2S2(soup)
# Save to a pandas DataFrame
IC2S2_names_df = pd.DataFrame(IC2S2_names, columns=["name"])
IC2S2_names_df.to_csv("IC2S2_names.csv", index=False)

### Q5: How many unique researchers do you get?
>- 1491

In [5]:
print(len(pd.read_csv("IC2S2_names.csv")))

1491


### Q6: Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices **(answer in max 150 words)**.

>- By inspecting the webpage, we were able to figure out that names were always contained in an &lt;a&gt; element for displaying the name properly. This means that finding an &lt;a&gt; element within one of the tables containing schedules would guarantee a name. When finding other names, like the ones that have the "chair", correctness was ensured by splitting that part from the name, ensuring only the name is retrieved.
>- Before adding a new name to the list, there is a check making sure that the name is not already in the found names list before adding it, making sure only unique names are in the list. The names contain no unwanted whitespace by calling the $\texttt{str.strip()}$ method before adding.

## Part 2: Ready Made vs Custom Made Data

### Q1: What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book **(answer in max 150 words)**.

>- 

## Part 3: Gathering Research Articles using the OpenAlex API


## Part 4: The Network of Computational Social Scientists

### Constructing the Computational Social Scientists Network

### Part 4.1: Network Construction

In [6]:
# Load the papers dataset
papers_df = pd.read_csv("../../comsocsci2024/lectures/papers.csv")
# Load the author's dataset
authors_df = pd.read_csv("../../comsocsci2024/lectures/authors.csv")
# Load the abstracts dataset
abstracts_df = pd.read_csv("../../comsocsci2024/lectures/abstracts.csv")

In [8]:
# Create weighted edgelist WIP
def create_weighted_edgelist(paper_authors_ids, author_ids):
    edges = []
    pairs = []
    # Loop through the paper authors
    for authors in tqdm(paper_authors_ids, desc="Progress"):
        for i in range(len(authors)):
            for j in range(i + 1, len(authors)):
                pair = (authors[i], authors[j])
                if pair not in pairs and (pair[1], pair[0]) not in pairs:
                    co_authored_works_count = 0
                    pairs.append(pair)
                    for authors_ in paper_authors_ids:
                        if pair[0] in authors_ and pair[1] in authors_:
                            co_authored_works_count += 1
                    edges.append((pair[0], pair[1], co_authored_works_count))
    
    return edges

papers_authors_ids = papers_df["author_ids"].apply(ast.literal_eval)
author_ids = authors_df["id"]
print(create_weighted_edgelist(papers_authors_ids, author_ids))
#for index, row in papers_df.iterrows():


Progress:   1%|          | 249/37523 [00:58<2:25:55,  4.26it/s] 


KeyboardInterrupt: 